In [2]:
import time
import torch
import torch.utils.data as data
from torch.autograd import Variable
from ssd import build_ssd
from layers.modules import MultiBoxLoss
from utils.augmentations import SSDAugmentation
from data import *

In [3]:
num_classes = 2
min_dim = 300
batch_size = 4
lr = 1e-4
gamma = 0.1


In [4]:
ssd_net = build_ssd('train', min_dim, num_classes)
vgg_weights = torch.load('weights/vgg16_reducedfc.pth')
ssd_net.vgg.load_state_dict(vgg_weights)

<All keys matched successfully>

In [5]:
# optimizer = torch.optim.Adam(ssd_net.parameters())
optimizer = torch.optim.Adadelta(ssd_net.parameters())
criterion = MultiBoxLoss(num_classes, 0.5, True, 0, True, 3, 0.5,
                            False, False)

In [6]:
train_dataset = CustomDataset(data_dict_file="./data/seminar_objdet_retina_oi5_ball/oi5_ball_filename_to_bbox_train.json",
                        transform=SSDAugmentation(min_dim, MEANS),
                        add_path = './data/seminar_objdet_retina_oi5_ball/')
train_data_loader = data.DataLoader(train_dataset, batch_size,
                                    shuffle=True, collate_fn=detection_collate,
                                    pin_memory=True)

In [7]:
ssd_net.train()
# loss counters
loc_loss = 0
conf_loss = 0
epoch = 0
start_iter = 0
step_index = 0
# lr_steps = (80000, 100000, 120000)
epoch_size = len(train_dataset) // batch_size
max_iter = epoch_size

In [8]:
epoch_size

718

In [34]:
# def adjust_learning_rate(optimizer, gamma, step):
#     """Sets the learning rate to the initial LR decayed by 10 at every
#         specified step
#     # Adapted from PyTorch Imagenet example:
#     # https://github.com/pytorch/examples/blob/master/imagenet/main.py
#     """
#     lr = lr * (gamma ** (step))
#     for param_group in optimizer.param_groups:
#         param_group['lr'] = lr

In [9]:
batch_iterator = iter(train_data_loader)
for iteration in range(start_iter, max_iter):

    # if iteration in lr_steps:
    #     step_index += 1
    #     adjust_learning_rate(optimizer, gamma, step_index)

    # load train data
    images, targets = next(batch_iterator)

    images = images
    # for ann in targets:
    #      ann.requires_grad_(False)

    # forward
    t0 = time.time()
    out = ssd_net(images)
    # backprop
    optimizer.zero_grad()
    loss_l, loss_c = criterion(out, targets)
    loss = loss_l + loss_c
    loss.backward()
    optimizer.step()
    t1 = time.time()
    loc_loss += loss_l.data.item()
    conf_loss += loss_c.data.item()

    if iteration % 10 == 0:
            print('iter ' + repr(iteration) + ' || Loss: %.4f ||' % (loss.data.item()), end=' ')
            print('timer: %.4f sec.' % (t1 - t0))


    if iteration != 0 and iteration % epoch_size == 0:
        print('Saving state, iter:', iteration)
        torch.save(ssd_net.state_dict(), 'weights/ssd300_custom_dataset_' +
                    repr(iteration) + '.pth')
torch.save(ssd_net.state_dict(), 'weights/custom_dataset.pth')

/home/vlad/.venv/lib/python3.11/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


iter 0 || Loss: 68124.9688 || timer: 3.1815 sec.


KeyboardInterrupt: 

In [40]:
ssd_net.train(False)

SSD(
  (vgg): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, cei

In [43]:
with torch.no_grad():
    batch_iterator = iter(train_data_loader)
    sum_loss = 0
    k = 0
    for iteration in range(start_iter, max_iter):

        # if iteration in lr_steps:
        #     step_index += 1
        #     adjust_learning_rate(optimizer, gamma, step_index)

        # load train data
        images, targets = next(batch_iterator)

        images = images
        # for ann in targets:
        #      ann.requires_grad_(False)

        # forward
        out = ssd_net(images)
        # backprop
        loss_l, loss_c = criterion(out, targets)
        loss = loss_l + loss_c
        sum_loss += loss
        k += 1
        

        if iteration % 10 == 0:
                print('iter ' + repr(iteration) + ' || Loss: %.4f ||' % (loss.data.item()), end='\n')


iter 0 || Loss: 78407.6875 ||
iter 10 || Loss: 53167.5117 ||
iter 20 || Loss: 64003.1133 ||
iter 30 || Loss: 23513.1660 ||
iter 40 || Loss: 45187.4883 ||
iter 50 || Loss: 57627.9219 ||
iter 60 || Loss: 34686.5273 ||
iter 70 || Loss: 45935.8672 ||
iter 80 || Loss: 41345.1484 ||
iter 90 || Loss: 65956.6875 ||
iter 100 || Loss: 87991.6484 ||
iter 110 || Loss: 56131.4219 ||
iter 120 || Loss: 70572.7500 ||
iter 130 || Loss: 69529.1172 ||
iter 140 || Loss: 99090.1172 ||
iter 150 || Loss: 59301.9023 ||
iter 160 || Loss: 71552.2656 ||
iter 170 || Loss: 59502.6094 ||


KeyboardInterrupt: 